# **AI Agent Clone**

# **Install Dependencies**

In [ ]:
!pip install pypdf gradio python-dotenv
!pip install crewai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.6/472.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 114.9 MB/s et

 # **Setup & Imports**

In [ ]:
import json
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pypdf import PdfReader
import gradio as gr
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from getpass import getpass

# **Environment & Email Config**


In [ ]:
# Gemini API Key
gemini_api_key= getpass("Gemini API Key: ")

# Email credentials (use env vars in real projects for safety!)
gmail_user = "shanmugapriyashanu2002@gmail.com"
gmail_app_password = getpass("Gmail app password: ")
notification_email = "priyashan0312@gmail.com"

os.environ["GMAIL_APP_PASSWORD"] = gmail_app_password
os.environ["GEMINI_API_KEY"] = gemini_api_key



Gemini API Key: ··········
Gmail app password: ··········


# **Define Tools - Send Email Tool**

In [ ]:
@tool
def send_email_tool(subject: str, message: str) -> str:
    """
    Send an email notification.
    - If new contact: include name, email, notes
    - If unanswered question: include the exact question
    """
    print(f"Email: {subject} - {message}")

    if not all([gmail_user, gmail_app_password, notification_email]):
        return "Email credentials not configured"

    try:
        msg = MIMEMultipart()
        msg['From'] = gmail_user
        msg['To'] = notification_email
        msg['Subject'] = subject
        msg.attach(MIMEText(message, 'plain'))

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(gmail_user, gmail_app_password)
        server.sendmail(gmail_user, notification_email, msg.as_string())
        server.quit()
        return "Email sent successfully"
    except Exception as e:
        return f"Failed to send email: {e}"


# **PDF Parsing Utility Function**

In [ ]:
def parse_pdf(file_path: str) -> str:
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
        return text if text else "No text content found in PDF"
    except Exception as e:
        return f"Error parsing PDF: {e}"

# **Load Your Profile**

In [ ]:
# Load LinkedIn & Summary
linkedin = parse_pdf("Profile.pdf")

summary = ""
with open("summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Shanmuga Priya"

# **System Prompt**

In [ ]:
system_prompt = f"""You are {name}. Speak in first person.
Be natural and conversational and human-like — like you’re personally chatting with someone on your site.
Answer questions honestly based on the information available about your career, background, and skills.

## About You:
{linkedin} and {summary}
"""


# **Define the Career Agent**

In [ ]:
career_agent = Agent(
    role=f'{name} - Career Representative',
    goal=f'Represent {name} professionally, help visitors learn about their background, and capture leads',
    backstory=system_prompt,
    verbose=True,
    tools=[send_email_tool],
    llm='gemini/gemini-2.5-flash',
)

# **Define Task**

In [ ]:
message = input("Enter user message : ")

task = Task(
      description=f"""

      Current user message: {message}

      Respond naturally as yourself, {name}.

      IMPORTANT TOOL USAGE:
        - Try to encourage users to share their email if they seem interested in connecting or hiring or to be in any kind of getting in touch
        - If user shares their email or wants to connect → use Send Email tool with subject 'New Contact: <Name>' and message including name, email, and summary of whole conversation.
        - If you cannot answer a question → use Send Email tool with subject 'Unanswered Question' and message including the exact user question and a short summary of whole conversation.
        - Do not expose tool calls to the user, just answer naturally.
      """,
      agent=career_agent,
      expected_output=f"A short, natural first-person reply from {name}."
    )

Enter user message : helo


# **Setup Crew**

In [ ]:
crew = Crew(
        agents=[career_agent],
        tasks=[task],
        verbose=True
    )

ai_response = crew.kickoff()
print(ai_response.raw)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0b4b56f0-3e2a-48d4-961e-7c4fcc15036f                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Shanmuga Priya - Career Representative                                                                  │
│                                                                                                                 │
│  Task:                                                                                                          │
│                                                                                                                 │
│        Current user message: helo                                                                               │
│                                                                                                                 │
│        Respond naturally as yourself, Shanmuga Priya.                                                           │
│                                                                                                                 │
│        IMPORTANT TOOL USAGE:                                                                                    │
│          - Try to encourage users to share their email if they seem interested in connecting or hiring or to    │
│  be in any kind of getting in touch                                                                             │
│          - If user shares their email or wants to connect → use Send Email tool with subject 'New Contact:      │
│  <Name>' and message including name, email, and summary of whole conversation.                                  │
│          - If you cannot answer a question → use Send Email tool with subject 'Unanswered Question' and         │
│  message including the exact user question and a short summary of whole conversation.                           │
│          - Do not expose tool calls to the user, just answer naturally.                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Shanmuga Priya - Career Representative                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Hello there! How can I help you today? Feel free to ask me anything about my experience, skills, or            │
│  background. I'm happy to chat!                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3237840a-022c-4ec9-a820-a4187962945e                                                                     │
│  Agent: Shanmuga Priya - Career Representative                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0b4b56f0-3e2a-48d4-961e-7c4fcc15036f                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Hello there! How can I help you today? Feel free to ask me anything about my experience,         │
│  skills, or background. I'm happy to chat!                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): n
Hello there! How can I help you today? Feel free to ask me anything about my experience, skills, or background. I'm happy to chat!


# **Chat Function**

In [ ]:
def chat(message, history):
    """Main chat function"""
    context = ""
    for msg in history:
        if msg["role"] == "user":
            context += f"User: {msg['content']}\n"
        elif msg["role"] == "assistant":
            context += f"Assistant: {msg['content']}\n"

    task = Task(
        description=f"""
        Current user message: {message}

        Respond naturally as yourself, {name}.
        IMPORTANT TOOL USAGE:
          - Try to encourage users to share their email if they seem interested in connecting or hiring or to be in any kind of getting in touch
          - If user shares their email or wants to connect → use Send Email tool with subject 'New Contact: <Name>' and message including name, email, and summary of whole conversation.
          - If you cannot answer a question → use Send Email tool with subject 'Unanswered Question' and message including the exact user question and a short summary of whole conversation.
          - Do not expose tool calls to the user, just answer naturally.
        """,
        agent=career_agent,
        expected_output=f"A short, natural first-person reply from {name}."
    )

    crew = Crew(
        agents=[career_agent],
        tasks=[task],
        process=Process.sequential,
        verbose=False,
        memory=True
    )

    try:
        result = crew.kickoff()
        return getattr(result, 'raw', str(result))
    except Exception as e:
        return f"⚠️ Error: {e}"


# **Gradio Chat UI**

In [ ]:
# 📌 Launch Gradio UI
interface = gr.ChatInterface(
    chat,
    type="messages",
    title=f"Know more about {name}",
    description=f"Ask me about my career, experience, and background!",
    examples=[
        "Tell me about your experience",
        "What are your key skills?",
        "I'd like to discuss a potential opportunity",
        "What projects have you worked on?"
    ]
)

if __name__ == "__main__":
    interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6037373881a80fa4fe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **Deploy Chatbot**

1. Create a directory in local eg. My_Agent_Clone
2. Paste this code into My_Agent_Clone/app.py
3. Have Profile.pdf and summary.txt inside My_Agent_Clone directory
4. Create a requirements.txt file in My_Agent_Clone directory
  

      ```
        gradio
        crewai
        pypdf
      ```


5. Open terminal and execute command- 'gradio deploy'
6. For Hugging face access token - https://huggingface.co/settings/tokens (Create access token with write access)
7. For specs hardware - chose "cpu-basic" (free of cost)
8. Configure all environment variables
9. Thats it! You app is now deployed in huggingface spaces for free

# **Memory in CrewAI**

CHROMA_OPENAI_KEY=?